# REBER Grammar with RNN

This workbook follows the notebook regarding <a href="https://www.willamette.edu/~gorr/classes/cs449/reber.html" target="_blank">Reber's grammar</a> words. In this one we gonna train a classifier to validate Embedded Reber's word

## What is a Reber Word ?

The embedded version Reber word is a word following the graph:

<img src="embreber.gif"/>

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import time
import random

import create_dataset as reber

from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM, SimpleRNN, GRU, TimeDistributed,Activation
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence

np.set_printoptions(precision=3)
np.set_printoptions(suppress=True)

%matplotlib inline

Using TensorFlow backend.


## Preparation of datas

We discover on the previous notebook how to generate a dataset for the training. We just gonna change it to use Embedded Word

In [2]:
x, y = reber.get_one_embedded_example(minLength=10)
print(reber.sequenceToWord(x))

BTBTSSSXXVPSET


In [3]:
reber.in_grammar(reber.sequenceToWord(x)[2:-1])

True

In [4]:
print(reber.get_char_one_hot("B"))
print(reber.sequenceToWord(x))

[array([ 1.,  0.,  0.,  0.,  0.,  0.,  0.])]
BTBTSSSXXVPSET


In [6]:
def set_wrong(good_seq):
    index = np.random.randint(len(good_seq))
    letter = reber.sequenceToWord([good_seq[index]])
    new_letter = random.choice(list(set("BTSXPVE") - set(letter)))
    bad_seq = good_seq.copy()
    bad_seq[index] = np.array(reber.get_char_one_hot(new_letter)[0])
    return bad_seq

x, y = reber.get_one_embedded_example(minLength=10)
print(reber.sequenceToWord(x))
x2 = set_wrong(x)
print(reber.sequenceToWord(x2))

BPBPTVPXVPXTVPSEP
BPBPXVPXVPXTVPSEP


To generate the target, we can reuse the previous fonction to generate the output based on the input

So now let's build our dataset. Due to the embedding features, the maxlen for the padding will be increase to 30

In [31]:
maxlen = 30
min_length = 10

X_train, y_train = [], []
X_test, y_test = [], []
X_val, y_val = [], []
y_possible = []

for i in range(1000):
    x, y = reber.get_one_embedded_example(minLength=min_length)
    res = [[1]]*30
    if random.random() < 0.5:
        x = set_wrong(x)
        res = [[0]]*30
    X_train.append(x)
    y_train.append(res)

for i in range(200):
    x, y = reber.get_one_embedded_example(minLength=min_length)
    res = [[1]]*30
    if random.random() < 0.5:
        x = set_wrong(x)
        res = [[0]]*30
    X_test.append(x)
    y_test.append(res)  
    
X_train = np.array(X_train)
y_train = np.array(y_train)
X_test = np.array(X_test)
y_test = np.array(y_test)

X_train = sequence.pad_sequences(X_train, maxlen=maxlen, padding='post', truncating='post')
#y_train = sequence.pad_sequences(y_train, maxlen=maxlen)
X_test = sequence.pad_sequences(X_test, maxlen=maxlen, padding='post', truncating='post')
#y_test = sequence.pad_sequences(y_test, maxlen=maxlen)

print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(1000, 30, 7)
(1000, 30, 1)
(200, 30, 7)
(200, 30, 1)


In [32]:
#print(y_test)
print(np.sum(y_test, axis=0))

[[96]
 [96]
 [96]
 [96]
 [96]
 [96]
 [96]
 [96]
 [96]
 [96]
 [96]
 [96]
 [96]
 [96]
 [96]
 [96]
 [96]
 [96]
 [96]
 [96]
 [96]
 [96]
 [96]
 [96]
 [96]
 [96]
 [96]
 [96]
 [96]
 [96]]


Previously, we found out that the GRU performed really better than LSTM and SimpleRNN. As a result we will focus on this model to improve it to generate correct sequences

## Test on GRU

During the writing of this notebook, I tried some loss, metrics and optimizer. The following ones are the one fitting the best

In [33]:
nb_unit = 7
inp_shape = (maxlen, 7)
loss_ = "mean_squared_error"
metrics_ = "mean_squared_error"
optimizer_ = "Nadam"
nb_epoch = 1000
batch_size = 1024

In [34]:
model = Sequential()
model.add(LSTM(units=nb_unit, input_shape=inp_shape, return_sequences=True))  # single LSTM
model.add(TimeDistributed(Dense(1), input_shape=inp_shape))
model.compile(loss=loss_, optimizer=optimizer_, metrics=[metrics_])

In [35]:
print("Inputs: {}".format(model.input_shape))
print("Outputs: {}".format(model.output_shape))
print("Actual input: {}".format(X_train.shape))
print("Actual output: {}".format(y_train.shape))

Inputs: (None, 30, 7)
Outputs: (None, 30, 1)
Actual input: (1000, 30, 7)
Actual output: (1000, 30, 1)


In [36]:
start = time.time()
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=20, batch_size=500)
stop = time.time()
t = stop-start
print(model.summary(), end=" ")
print("Training time : {:}s".format(t))

Train on 1000 samples, validate on 200 samples
Epoch 1/20
1000/1000 [==============================] - 2s - loss: 0.4212 - mean_squared_error: 0.4212 - val_loss: 0.3769 - val_mean_squared_error: 0.3769
Epoch 2/20
1000/1000 [==============================] - 0s - loss: 0.4013 - mean_squared_error: 0.4013 - val_loss: 0.3628 - val_mean_squared_error: 0.3628
Epoch 3/20
1000/1000 [==============================] - 0s - loss: 0.3868 - mean_squared_error: 0.3868 - val_loss: 0.3496 - val_mean_squared_error: 0.3496
Epoch 4/20
1000/1000 [==============================] - 0s - loss: 0.3728 - mean_squared_error: 0.3728 - val_loss: 0.3368 - val_mean_squared_error: 0.3368
Epoch 5/20
1000/1000 [==============================] - 0s - loss: 0.3593 - mean_squared_error: 0.3593 - val_loss: 0.3245 - val_mean_squared_error: 0.3245
Epoch 6/20
1000/1000 [==============================] - 0s - loss: 0.3462 - mean_squared_error: 0.3462 - val_loss: 0.3129 - val_mean_squared_error: 0.3129
Epoch 7/20
1000/1000 [=

In [37]:
y_pred = model.predict(X_test)
print(y_pred)

[[[ 0.181]
  [ 0.322]
  [ 0.422]
  ..., 
  [ 0.393]
  [ 0.384]
  [ 0.376]]

 [[ 0.181]
  [ 0.357]
  [ 0.455]
  ..., 
  [ 0.387]
  [ 0.379]
  [ 0.371]]

 [[ 0.181]
  [ 0.357]
  [ 0.455]
  ..., 
  [ 0.357]
  [ 0.353]
  [ 0.348]]

 ..., 
 [[ 0.181]
  [ 0.357]
  [ 0.455]
  ..., 
  [ 0.369]
  [ 0.363]
  [ 0.357]]

 [[ 0.181]
  [ 0.322]
  [ 0.422]
  ..., 
  [ 0.377]
  [ 0.369]
  [ 0.363]]

 [[ 0.181]
  [ 0.357]
  [ 0.455]
  ..., 
  [ 0.375]
  [ 0.368]
  [ 0.362]]]


In [38]:
score, acc = model.evaluate(X_test, y_test, batch_size=1)
print('Test score:', score)
print('Test accuracy:', acc)

200/200 [==============================] - 1s     
Test score: 0.257494045272
Test accuracy: 0.257494045272


## Evaluation

Every models will be evaluated on the fonction designed previously which count on 20 x 100 word generated by the NN, how much are following the rule. But first, let's check if the training is "over" by checking the loss

In [ ]:
plt.plot(history.history["loss"], label="GRU")
plt.show()

The loss is "stable" even if it didn't reached yet the best point. We can also take a look to the output based on the X_val we generated

In [ ]:
print("Input :")
print(X_val)
print("\n\n Output :")
y_pred = model.predict(X_val)
print(y_pred)

We can also perform the cleaning and compare it to the expected output

In [ ]:
y_pred = np.where(y_pred < 0.1, 0, y_pred)

In [ ]:
for pred, real in zip(y_pred[0], y_possible[0]):
    print(pred, "\t", real)

We can see that the output is clearly more "shuffled". This model starts to show it's own limit. We can check the output on the generation of words

In [ ]:
def is_embedded_word(w):
    if w[:2] not in ["BT", "BP"]:
        return False
    if reber.in_grammar(w[2:-1]):
        return False
    if w[-1] not in ["T", "P"]:
        return False
    return True

def Pick_From_Output(x):
    y = np.zeros_like(x)
    x = np.where(x < 0.1, 0, x)
    x = x[0]/x[0].sum(axis=1)
    i = np.random.choice(list(range(7)), size=1, p=x[0])
    y[0,0,i] = 1
    return y

def evaluate(model, nb_word = 1, max_iter = 50):
    good_pred = 0
    for _ in range(nb_word):
        model.reset_states()
        first_input = np.array([[[1,0,0,0,0,0,0]]])
        word = "B"
        loop = 0
        nextLetter = "B"
        next_seq = first_input
        while nextLetter != "E" and loop < max_iter:
            y_pred = model.predict(next_seq)
            next_seq = Pick_From_Output(y_pred)
            nextLetter = reber.sequenceToWord(next_seq[0])
            loop += 1
            word += nextLetter
        if is_embedded_word(word):
            good_pred += 1
    acc = 100*good_pred/nb_word
    print("Good prediction : {:.2f}%".format(acc))
    return acc

In [ ]:
newModel = Sequential()
newModel.add(GRU(units=7, stateful=True, batch_input_shape=(1,1,7), return_sequences=True, verbose=0))
newModel.set_weights(model.get_weights())

In [ ]:
result_GRU = []
for _ in range(20):
    result_GRU.append(evaluate(newModel, 100, 50))

The output is really worse than it was previously. This is due to the more complexe rule behind the embedded words

In [ ]:
x = list(range(20))
y = [result_GRU]
labels = ["GRU"]

plt.figure(figsize=(12, 12))
for y_arr, label in zip(y, labels):
    plt.plot(x, y_arr, label=label)

plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.show()

## Improvements

Now let's do the same but with differents topology and features

In [ ]:
nb_unit = 7
inp_shape = (maxlen, 7)
loss_ = "mean_squared_error"
metrics_ = "mean_squared_error"
optimizer_ = "Nadam"
nb_epoch = 1000
batch_size = 1024

In [ ]:
model = Sequential()
model.add(GRU(units=nb_unit, input_shape=inp_shape, return_sequences=True))
model.add(Dropout(0.2))
model.add(GRU(units=nb_unit, return_sequences=True))
model.compile(loss=loss_,
              optimizer=optimizer_,
              metrics=[metrics_])

In [ ]:
start = time.time()
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=nb_epoch, batch_size=batch_size, verbose=0)
stop = time.time()
t = stop-start
print(model.summary(), end=" ")
print("Training time : {:}s".format(t))
plt.plot(history.history["loss"], label="GRU")

In [ ]:
newModel = Sequential()
newModel.add(GRU(units=7, stateful=True, batch_input_shape=(1,1,7), return_sequences=True))
newModel.add(GRU(units=nb_unit, return_sequences=True))
newModel.set_weights(model.get_weights())

result_GRU = []
for _ in range(20):
    result_GRU.append(evaluate(newModel, 100, 50))

plt.plot(list(range(20)), result_GRU)
plt.show()

## Conclusion

IUn this workbook, we started to go through RNN. We check simple model of both LSTM, GRU and SimpleRNN to check how fast and well they learn. On this example GRU outperform other models for 2 reasons:
<li>LSTM are better for long sequence memory. On this short example, the generator jumped from a node to another one with the same letter (see red arrow below) <img src="reber_jump.png"/>. In fact, it outputs a too high probability of those non-allowed rules and the "PickOne" function had risks to pick it</li>
<li>Simple RNN are not strong enougth with 1 hidden layer to "remember" all those rules. We need a longer NN which is also longer to train. This simpleRNN is faster but nearly never used anymore as it perform very poorly on lot of cases</li>

## Going further

On a future notebook, we will explore Embedded Reber but using deeper RNNs